In [1]:
!python -V

Python 3.9.12


In [2]:
import pandas as pd

In [3]:
import pyarrow

In [4]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression

from sklearn.metrics import mean_squared_error

### Question 1 Downloading the Data

We'll use the same NYC taxi dataset, but instead of "Green Taxi Trip Records", we'll use "Yellow Taxi Trip Records".

Download the data for January and February 2023.

Read the data for January. How many columns are there?

In [5]:
# Read in the Jan 2023 data for yellow taxi trips
df = pd.read_parquet('./data/yellow_tripdata_2023-01.parquet', engine='pyarrow')

# How many columns of data?
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3066766 entries, 0 to 3066765
Data columns (total 19 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   VendorID               int64         
 1   tpep_pickup_datetime   datetime64[ns]
 2   tpep_dropoff_datetime  datetime64[ns]
 3   passenger_count        float64       
 4   trip_distance          float64       
 5   RatecodeID             float64       
 6   store_and_fwd_flag     object        
 7   PULocationID           int64         
 8   DOLocationID           int64         
 9   payment_type           int64         
 10  fare_amount            float64       
 11  extra                  float64       
 12  mta_tax                float64       
 13  tip_amount             float64       
 14  tolls_amount           float64       
 15  improvement_surcharge  float64       
 16  total_amount           float64       
 17  congestion_surcharge   float64       
 18  airport_fee           

### Q2. Computing duration

Now let's compute the duration variable. It should contain the duration of a ride in minutes.

What's the standard deviation of the trips duration in January?

In [6]:
df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
df.duration = df.duration.apply(lambda td: td.total_seconds() / 60)

df.duration.std()

42.594351241920904

### Q3. Dropping outliers

Next, we need to check the distribution of the duration variable. There are some outliers. Let's remove them and keep only the records where the duration was between 1 and 60 minutes (inclusive).

What fraction of the records left after you dropped the outliers?

In [7]:
# Drop outliers
df = df[(df.duration >= 1) & (df.duration <= 60)]

# Find fraction of records remaining
len(df)/3066766

0.9812202822125979

### Q4. One-hot encoding

Let's apply one-hot encoding to the pickup and dropoff location IDs. We'll use only these two features for our model.

* Turn the dataframe into a list of dictionaries (remember to re-cast the ids to strings - otherwise it will label encode them)
* Fit a dictionary vectorizer
* Get a feature matrix from it

What's the dimensionality of this matrix (number of columns)?

In [8]:
# Using only 2 features for the model: pickup and dropoff location IDs
categorical = ['PULocationID', 'DOLocationID']

# Change location IDs from int to str
df[categorical] = df[categorical].astype(str)

In [9]:
# Fit a dictionary vectorizer and get a feature matrix

train_dicts = df[categorical].to_dict(orient='records')

dv = DictVectorizer()
X_train = dv.fit_transform(train_dicts)

X_train.shape

(3009173, 515)

### Q5. Training a model

Now let's use the feature matrix from the previous step to train a model.

Train a plain linear regression model with default parameters.

Calculate the RMSE of the model on the training data. 
What's the RMSE on train?

In [10]:
target = 'duration'
y_train = df[target].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_train)

mean_squared_error(y_train, y_pred, squared=False)

7.6492610279057605

### Q6. Evaluating the model

Now let's apply this model to the validation dataset (February 2023).

What's the RMSE on validation?

In [11]:
# read the Feb 2023 as validation dataset
df_val = pd.read_parquet('./data/yellow_tripdata_2023-02.parquet', engine='pyarrow')

# Change location IDs from int to str
df_val[categorical] = df_val[categorical].astype(str)

In [12]:
# Compute the validation dataset duration

df_val['duration'] = df_val.tpep_dropoff_datetime - df_val.tpep_pickup_datetime
df_val.duration = df_val.duration.apply(lambda td: td.total_seconds() / 60)

# Drop outliers
df_val = df_val[(df_val.duration >= 1) & (df_val.duration <= 60)]

In [13]:
# Get the validation feature matrix from the trained Dict Vectorizer

val_dicts = df_val[categorical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

X_val.shape

(2855951, 515)

In [14]:
# Apply the trained model to the validation dataset

target = 'duration'
y_val = df_val[target].values

y_pred = lr.predict(X_val)

mean_squared_error(y_val, y_pred, squared=False)

7.81183265470218